Download Packages

In [1]:

!pip install sentence-transformers
!pip install numpy
!pip install flask
!pip install python-dotenv

Importing Libraries

In [2]:
import os
import sys
import re
import numpy as np
import requests
from flask import Flask, render_template, request, redirect, url_for
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

c:\Users\User\Downloads\Compressed\GeoSpeak Project\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Get Conests

In [3]:
load_dotenv()
secret_key = os.getenv('SECRET_KEY')
vector_database = os.getenv('VECTOR_DATABASE')
model_name = os.getenv('MODEL_NAME')

Download the model

In [4]:
model = SentenceTransformer(model_name)

c:\Users\User\Downloads\Compressed\GeoSpeak Project\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Application Classes

Flask Class

In [5]:
# app = Flask(__name__)
# at run app

Message Class

In [6]:
class ErrorMessages:
    def __init__(self):
        self.message = {
            "English" : "Please enter text in English only (including symbols and numbers).",
            "show_details" :"Please translate first.",
            "Trans_in_server":"Error occurred while Translating",
            "Connecting_to_server":"Error occurred while Connecting",
            "can not show" :"can not show details because server error",
            "length" :"The text must be between 2 and 2000",
        }

messages = ErrorMessages()

Embedding Class

In [7]:
class Embedding:
    def __init__(self):
        pass
    def store_embeddings_to_file(self, texts, filename):
        with open(filename, 'w') as file:
            for i, text in enumerate(texts):
                embedding = model.encode(text.lower()).astype('float32')
                embedding_str = ','.join(map(str, embedding))
                file.write(f"{text}\t{embedding_str}\n")
                print(f"\nthis the {i+1} time!\n")
                print(f"Embedding for '{text}': {embedding}")
                print(f"Embedding size for '{text}': {embedding.size}")

    def load_embeddings_from_file(self, filename):
        embeddings = {}
        with open(filename, 'r') as file:
            for line in file:
                text, embedding_str = line.strip().split('\t')
                embedding = np.array(list(map(float, embedding_str.split(','))), dtype=np.float32)
                embeddings[text] = embedding
        return embeddings
    
    def query_vector_db(self, embedding, embeddings, top_k=5):
        distances = {}
        for text, stored_embedding in embeddings.items():
            distance = np.linalg.norm(embedding - stored_embedding)
            distances[text] = distance
        sorted_texts = sorted(distances, key=distances.get)
        return [(text, distances[text]) for text in sorted_texts[:top_k]]

embedding_manager = Embedding()

Dealing with variable

In [8]:
class Variables():
    def __init__(self):
        self.data = None
        self.text = ""
        self.translated_text = None
        self.details = None
        self.current_details = None
        self.input_text = ""
        self.error = None
        self.lang = "ar"
var = Variables()

Translation Class

In [9]:
class Translation:
    def __init__(self):
        pass

    def create_context_aware_prompt(self, the_text, to_language, context):
        prompt = f"Roles: Don't return anything except the translated text\n Based on these samples: {', '.join(context)}, translate this text to {to_language} the following text: [{the_text}]"
        print(prompt)
        return prompt

    def translate_with_context(self, the_text, to_language):
        embeddings = embedding_manager.load_embeddings_from_file(vector_database)
        embedding = model.encode(the_text).astype('float32')
        similar_examples_data = embedding_manager.query_vector_db(embedding, embeddings)
        similar_examples = [text for text, _ in similar_examples_data]
        context_aware_prompt = self.create_context_aware_prompt(the_text, to_language, similar_examples)
        url = "https://cheapest-gpt-4-turbo-gpt-4-vision-chatgpt-openai-ai-api.p.rapidapi.com/v1/chat/completions"
        payload = {
            "messages": [{"role": "user", "content": context_aware_prompt}],
            "model": "gpt-4o",
            "max_tokens": 100,
            "temperature": 0.9
        }
        headers = {
            "x-rapidapi-key": secret_key,
            "x-rapidapi-host": "cheapest-gpt-4-turbo-gpt-4-vision-chatgpt-openai-ai-api.p.rapidapi.com",
            "Content-Type": "application/json"
        }
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code != 200:
            return [f"{messages.message['Connecting_to_server']}: {response.status_code} {response.text}",messages.message["can not show"]]

        try:
            formated_text = ', '.join(similar_examples)
            return [response.json()["choices"][0]["message"]["content"], formated_text]
        except KeyError:
            return [messages.message["Trans_in_server"],messages.message["can not show"]]
        
    def validate_input(self, text):
        return bool(re.match("[\w\s.,!?'\"-& % # @ ^ * : $; / - + = ~ ]", text))

application = Translation()

Data and Vector Database

In [10]:
data_emb = [
    "What do you think?",
    "Are you sure?",
    "Then you have to go out",
    "My name is",
    "That's so funny",
    "Okay then",
    "You see?",
    "How are you?",
    "I am fine, thank you.",
    "Where are you from?",
    "I am from New York.",
    "What time is it?",
    "It's 3 o'clock.",
    "Can you help me?",
    "Sure, how can I help?",
    "Do you speak English?",
    "Yes, a little.",
    "What is your favorite color?",
    "My favorite color is blue.",
    "How old are you?",
    "I am 25 years old.",
    "Do you have any pets?",
    "Yes, I have a cat.",
    "What is your favorite food?",
    "I love pizza.",
    "Where do you live?",
    "I live in London.",
    "What is your job?",
    "I am a teacher.",
    "Can I have the bill, please?",
    "Sure, here it is.",
    "How much does this cost?",
    "It costs twenty dollars.",
    "What is the weather like?",
    "It's sunny and warm.",
    "Is it going to rain?",
    "No, it's clear today.",
    "Where is the nearest station?",
    "It's just around the corner.",
    "How do I get to the airport?",
    "Take the train to the airport.",
    "Can I borrow your pen?",
    "Of course, here you go.",
    "Do you have a map?",
    "Yes, I have one.",
    "Where can I buy a ticket?",
    "You can buy a ticket at the counter.",
    "What is your favorite movie?",
    "My favorite movie is Inception.",
    "Do you like reading books?",
    "Yes, I love reading.",
    "What are you doing?",
    "I am studying for an exam.",
    "Can you recommend a restaurant?",
    "Sure, try the Italian place on Main Street.",
    "How was your day?",
    "It was great, thanks for asking.",
    "Do you have any siblings?",
    "Yes, I have one brother and one sister.",
    "What time does the store close?",
    "It closes at 9 PM.",
    "Where can I find a taxi?",
    "There is a taxi stand across the street.",
    "Is there a hospital nearby?",
    "Yes, it's a few blocks away.",
    "How long does it take to get there?",
    "It takes about 15 minutes.",
    "Can I get a refund?",
    "Yes, if you have the receipt.",
    "Where is the nearest ATM?",
    "It's next to the supermarket.",
    "Do you have a reservation?",
    "Yes, I made a reservation under Smith.",
    "What is the password for the Wi-Fi?",
    "The password is 'welcome123'.",
    "Can you show me how to use this?",
    "Sure, let me explain.",
    "Is there a discount available?",
    "Yes, there's a 10% discount today.",
    "How can I contact customer service?",
    "You can call the number on the website.",
    "Where is the restroom?",
    "It's down the hall to the left.",
    "What is the best way to learn English?",
    "Practice speaking with native speakers.",
    "Can you recommend a good book?",
    "I recommend 'To Kill a Mockingbird'.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like your coffee?",
    "I like it with cream and sugar.",
    "Do you enjoy cooking?",
    "Yes, I enjoy trying new recipes.",
    "Where did you grow up?",
    "I grew up in Chicago.",
    "What kind of music do you like?",
    "I like classical music.",
    "Can you tell me a joke?",
    "Sure, why did the scarecrow win an award? Because he was outstanding in his field!",
    "What are your hobbies?",
    "I enjoy hiking and photography.",
    "Do you play any sports?",
    "Yes, I play soccer.",
    "What is your favorite season?",
    "I love autumn.",
    "How do you celebrate your birthday?",
    "I usually have a party with friends.",
    "Can you recommend a good movie?",
    "Try watching 'The Shawshank Redemption'.",
    "Do you have any plans for the weekend?",
    "I plan to go hiking.",
    "What is your favorite restaurant?",
    "I love the Thai place downtown.",
    "How often do you exercise?",
    "I exercise three times a week.",
    "What is your favorite type of cuisine?",
    "I enjoy Italian food.",
    "Can you help me with my homework?",
    "Yes, I can assist you.",
    "What is your dream job?",
    "My dream job is to be a writer.",
    "How do you spend your free time?",
    "I spend it reading and relaxing.",
    "Where do you usually go on vacation?",
    "I like to visit tropical beaches.",
    "What is your favorite book?",
    "My favorite book is '1984'.",
    "Can you recommend a good place to visit?",
    "You should visit Paris.",
    "How do you get to work?",
    "I take the subway.",
    "Do you have any pets?",
    "Yes, I have a dog and a rabbit.",
    "What is your favorite dessert?",
    "I love chocolate cake.",
    "Where do you like to shop?",
    "I like to shop at the mall.",
    "What is your favorite sport?",
    "My favorite sport is basketball.",
    "How do you usually celebrate New Year's Eve?",
    "I celebrate with fireworks and a party.",
    "Do you like traveling?",
    "Yes, I love exploring new places.",
    "What is your favorite drink?",
    "My favorite drink is lemonade.",
    "How often do you go to the gym?",
    "I go to the gym four times a week.",
    "What kind of movies do you enjoy?",
    "I enjoy action and adventure movies.",
    "Can you recommend a good TV show?",
    "Try watching 'Breaking Bad'.",
    "How do you relax after a long day?",
    "I relax by watching TV or reading a book.",
    "What is your favorite childhood memory?",
    "My favorite memory is going to the beach with my family.",
    "Do you like going to concerts?",
    "Yes, I enjoy live music.",
    "What is your favorite city?",
    "My favorite city is Tokyo.",
    "Can you help me find a good book?",
    "Sure, what genre do you like?",
    "Do you enjoy outdoor activities?",
    "Yes, I love hiking and camping.",
    "What is your favorite thing to do on weekends?",
    "I like to relax and catch up on sleep.",
    "How do you stay motivated?",
    "I set goals and remind myself of them.",
    "What is your favorite way to spend a rainy day?",
    "I like to stay indoors and watch movies.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you like to celebrate holidays?",
    "I celebrate with family and friends.",
    "What is your favorite type of weather?",
    "I like sunny and warm weather.",
    "Do you enjoy visiting museums?",
    "Yes, I like learning about history and art.",
    "What is your favorite animal?",
    "My favorite animal is the tiger.",
    "Can you recommend a good restaurant?",
    "Try the new sushi place on Elm Street.",
    "How do you prefer to travel?",
    "I prefer traveling by train.",
    "What is your favorite holiday tradition?",
    "My favorite tradition is decorating the Christmas tree.",
    "Do you enjoy playing board games?",
    "Yes, I like playing with family and friends.",
    "What is your favorite type of movie?",
    "I enjoy science fiction films.",
    "Can you recommend a good restaurant for lunch?",
    "Try the deli on Maple Avenue.",
    "What is your favorite thing to do in your free time?",
    "I enjoy painting and drawing.",
    "How do you like to spend your summer vacations?",
    "I like to go to the beach and travel.",
    "What is your favorite holiday?",
    "My favorite holiday is Halloween.",
    "Do you enjoy gardening?",
    "Yes, I find it very relaxing.",
    "What is your favorite type of music?",
    "I like jazz music.",
    "How do you stay healthy?",
    "I eat a balanced diet and exercise regularly.",
    "Do you have a favorite hobby?",
    "Yes, I love playing the guitar.",
    "What is your favorite TV show?",
    "My favorite show is 'Friends'.",
    "Can you recommend a good place to eat?",
    "Try the new Italian restaurant downtown.",
    "What is your favorite way to spend a weekend?",
    "I enjoy hiking and having a barbecue.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you like cooking at home?",
    "Yes, I enjoy trying new recipes.",
    "What is your favorite activity during the winter?",
    "I like skiing and drinking hot chocolate.",
    "How often do you visit your family?",
    "I visit them once a month.",
    "What is your favorite place to visit in your city?",
    "I like visiting the local park.",
    "Can you recommend a good book to read?",
    "I recommend 'The Great Gatsby'.",
    "What is your favorite season?",
    "I love spring.",
    "How do you like to spend your evenings?",
    "I like to relax and watch TV.",
    "Do you enjoy attending live performances?",
    "Yes, I enjoy concerts and theater.",
    "What is your favorite sport to watch?",
    "I enjoy watching soccer.",
    "How do you celebrate special occasions?",
    "I celebrate with a party or a special meal.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite way to unwind?",
    "I enjoy reading a good book.",
    "How do you like to celebrate your birthday?",
    "I like having a party with friends and family.",
    "Do you enjoy visiting new places?",
    "Yes, I love exploring new destinations.",
    "What is your favorite type of cuisine?",
    "I enjoy Asian cuisine.",
    "How do you stay fit?",
    "I go to the gym and play sports.",
    "Do you prefer mornings or evenings?",
    "I prefer evenings.",
    "What is your favorite type of dessert?",
    "I love cheesecake.",
    "How do you spend your weekends?",
    "I like to relax and catch up on hobbies.",
    "Do you enjoy outdoor sports?",
    "Yes, I like playing tennis and swimming.",
    "What is your favorite type of book?",
    "I enjoy mystery novels.",
    "How often do you travel?",
    "I travel several times a year.",
    "Do you like watching movies at home or in the theater?",
    "I prefer watching movies at home.",
    "What is your favorite childhood game?",
    "My favorite game was hide and seek.",
    "How do you like to celebrate your achievements?",
    "I like to treat myself to something special.",
    "What is your favorite family tradition?",
    "I love the family game night.",
    "Do you enjoy learning new languages?",
    "Yes, I find it fascinating.",
    "What is your favorite place to eat out?",
    "I enjoy dining at the seafood restaurant.",
    "How do you like to spend your holidays?",
    "I like to travel and explore new places.",
    "Do you have a favorite place to go for a walk?",
    "I enjoy walking in the nature reserve.",
    "What is your favorite way to relax?",
    "I like to listen to music and meditate.",
    "How do you celebrate your anniversary?",
    "I celebrate with a special dinner.",
    "Do you enjoy visiting historical sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to spend a rainy day?",
    "I like to stay indoors and read a book.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer coffee or tea?",
    "I prefer tea.",
    "What is your favorite holiday destination?",
    "My favorite destination is Paris.",
    "How do you like to spend your vacation?",
    "I like to relax and explore new places.",
    "What is your favorite type of movie?",
    "I enjoy romantic comedies.",
    "Do you have a favorite restaurant for dinner?",
    "I love dining at the steakhouse.",
    "How do you stay motivated?",
    "I set goals and reward myself for achievements.",
    "Do you enjoy attending festivals?",
    "Yes, I love the atmosphere.",
    "What is your favorite way to spend your spare time?",
    "I enjoy practicing a musical instrument.",
    "How do you celebrate special events?",
    "I like to host a party or go out to eat.",
    "What is your favorite form of exercise?",
    "I enjoy cycling.",
    "Do you like to travel by plane or by train?",
    "I prefer traveling by plane.",
    "What is your favorite weekend activity?",
    "I like to go hiking.",
    "How do you prefer to spend your free time?",
    "I like to watch movies or read books.",
    "Do you have a favorite vacation spot?",
    "I love going to the mountains.",
    "What is your favorite type of music?",
    "I enjoy classical music.",
    "How often do you exercise?",
    "I exercise three times a week.",
    "Do you enjoy cooking?",
    "Yes, I like experimenting with new recipes.",
    "What is your favorite way to relax after work?",
    "I like to watch TV or go for a walk.",
    "How do you celebrate the New Year?",
    "I celebrate with fireworks and a party.",
    "What is your favorite type of cuisine?",
    "I enjoy Mediterranean food.",
    "Do you like to go to the gym?",
    "Yes, I go regularly.",
    "How do you stay fit?",
    "I do cardio and strength training.",
    "Do you prefer reading books or watching movies?",
    "I prefer reading books.",
    "What is your favorite way to spend a day off?",
    "I enjoy visiting a museum or park.",
    "How do you celebrate your friends' birthdays?",
    "I like to organize a surprise party.",
    "What is your favorite type of exercise?",
    "I enjoy yoga and Pilates.",
    "Do you enjoy attending sports events?",
    "Yes, I like going to games.",
    "What is your favorite thing to do on a weekend?",
    "I love relaxing and spending time with family.",
    "How do you stay productive during the day?",
    "I make a schedule and stick to it.",
    "Do you like to visit art galleries?",
    "Yes, I enjoy viewing different artworks.",
    "What is your favorite way to unwind?",
    "I like to listen to soothing music.",
    "How do you celebrate your successes?",
    "I treat myself to something I like.",
    "Do you prefer working from home or in an office?",
    "I prefer working from home.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your summer?",
    "I enjoy going to the beach and traveling.",
    "Do you enjoy visiting parks?",
    "Yes, I like to go for walks in the park.",
    "What is your favorite type of movie?",
    "I enjoy documentaries.",
    "How often do you go to the theater?",
    "I go to the theater a few times a year.",
    "Do you like to cook at home or eat out?",
    "I enjoy cooking at home.",
    "What is your favorite way to spend an afternoon?",
    "I like to read or go for a bike ride.",
    "How do you stay focused at work?",
    "I take regular breaks and stay organized.",
    "Do you enjoy exploring new places?",
    "Yes, I love discovering new locations.",
    "What is your favorite type of music?",
    "I enjoy rock music.",
    "How do you celebrate the holidays?",
    "I celebrate with family gatherings.",
    "Do you like to try new foods?",
    "Yes, I love trying new dishes.",
    "What is your favorite season?",
    "I enjoy summer.",
    "How do you stay healthy?",
    "I eat a balanced diet and exercise regularly.",
    "Do you prefer relaxing at home or going out?",
    "I prefer relaxing at home.",
    "What is your favorite hobby?",
    "I enjoy painting.",
    "How do you celebrate your achievements?",
    "I celebrate with a nice dinner.",
    "Do you enjoy visiting new cities?",
    "Yes, I like exploring urban areas.",
    "What is your favorite activity on weekends?",
    "I enjoy hiking and outdoor activities.",
    "How do you like to spend a day off?",
    "I like to relax and do something fun.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy spending time outdoors.",
    "What is your favorite type of food?",
    "I love Mexican food.",
    "How do you stay motivated to exercise?",
    "I set goals and track my progress.",
    "Do you enjoy playing video games?",
    "Yes, I like playing strategy games.",
    "What is your favorite holiday?",
    "My favorite holiday is Easter.",
    "How do you like to relax on weekends?",
    "I enjoy lounging and watching movies.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite way to spend your free time?",
    "I like to read and go for walks.",
    "How do you celebrate special occasions?",
    "I celebrate with a party or special event.",
    "Do you enjoy visiting historical landmarks?",
    "Yes, I find them fascinating.",
    "What is your favorite type of book?",
    "I enjoy fantasy novels.",
    "How do you like to spend your evenings?",
    "I like to relax and watch TV.",
    "Do you like attending concerts?",
    "Yes, I enjoy live music.",
    "What is your favorite weekend activity?",
    "I enjoy going for a hike.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you like to cook or bake?",
    "I enjoy both cooking and baking.",
    "What is your favorite type of weather?",
    "I like sunny weather.",
    "How do you celebrate birthdays?",
    "I celebrate with friends and family.",
    "Do you enjoy going to museums?",
    "Yes, I like learning about art and history.",
    "What is your favorite outdoor activity?",
    "I love hiking and camping.",
    "How do you stay productive?",
    "I make lists and set deadlines.",
    "Do you like spending time with family?",
    "Yes, I enjoy family gatherings.",
    "What is your favorite type of exercise?",
    "I enjoy swimming and jogging.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and parties.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of movie?",
    "I enjoy action and adventure films.",
    "How do you like to spend your holidays?",
    "I like to travel or spend time with loved ones.",
    "Do you enjoy trying new recipes?",
    "Yes, I love experimenting in the kitchen.",
    "What is your favorite TV show?",
    "My favorite show is 'Breaking Bad'.",
    "How do you stay healthy?",
    "I eat well and exercise regularly.",
    "Do you enjoy attending events?",
    "Yes, I like going to festivals and fairs.",
    "What is your favorite way to unwind?",
    "I enjoy reading a good book.",
    "How do you celebrate special moments?",
    "I celebrate with special activities or treats.",
    "Do you like to visit new restaurants?",
    "Yes, I love trying new places to eat.",
    "What is your favorite type of cuisine?",
    "I enjoy Indian food.",
    "How do you stay active?",
    "I go for walks and exercise regularly.",
    "Do you prefer reading or watching TV?",
    "I prefer reading books.",
    "What is your favorite way to relax?",
    "I enjoy listening to music and meditating.",
    "How do you celebrate holidays?",
    "I celebrate with festive activities.",
    "Do you enjoy visiting parks?",
    "Yes, I like spending time outdoors.",
    "What is your favorite type of book?",
    "I enjoy science fiction novels.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new places.",
    "Do you enjoy playing sports?",
    "Yes, I like playing tennis and soccer.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay focused?",
    "I set goals and prioritize tasks.",
    "Do you prefer staying in or going out?",
    "I enjoy staying in and relaxing.",
    "What is your favorite hobby?",
    "I love gardening.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you like to visit new cities?",
    "Yes, I love exploring new urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy strength training.",
    "How do you spend your holidays?",
    "I like to travel and spend time with family.",
    "Do you enjoy visiting museums?",
    "Yes, I find them very interesting.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and outdoor adventures.",
    "How do you stay motivated?",
    "I set personal goals and track progress.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite type of cuisine?",
    "I enjoy Italian food.",
    "How do you celebrate special occasions?",
    "I celebrate with special events or outings.",
    "Do you enjoy playing board games?",
    "Yes, I love playing with family and friends.",
    "What is your favorite type of weather?",
    "I like mild and sunny weather.",
    "How do you stay healthy?",
    "I eat a balanced diet and exercise regularly.",
    "Do you enjoy trying new activities?",
    "Yes, I love trying new things.",
    "What is your favorite type of music?",
    "I enjoy electronic music.",
    "How do you like to spend your free time?",
    "I like to relax and read books.",
    "Do you prefer outdoor or indoor activities?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite holiday?",
    "My favorite holiday is New Year's Eve.",
    "How do you stay organized?",
    "I use a calendar and make to-do lists.",
    "Do you enjoy going to the theater?",
    "Yes, I like watching plays and musicals.",
    "What is your favorite way to relax after work?",
    "I enjoy reading or watching TV.",
    "How do you celebrate the end of the year?",
    "I celebrate with friends and family gatherings.",
    "Do you prefer staying in or going out for dinner?",
    "I like going out for dinner.",
    "What is your favorite type of movie?",
    "I enjoy comedy films.",
    "How do you like to spend your days off?",
    "I like to relax and do activities I enjoy.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite outdoor activity?",
    "I love hiking and cycling.",
    "How do you stay productive?",
    "I create a daily plan and stick to it.",
    "Do you prefer coffee or tea in the morning?",
    "I prefer coffee.",
    "What is your favorite hobby?",
    "I enjoy photography.",
    "How do you celebrate special occasions?",
    "I celebrate with family or friends.",
    "Do you like visiting art galleries?",
    "Yes, I enjoy viewing art exhibitions.",
    "What is your favorite way to unwind?",
    "I like to listen to calming music.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a healthy diet.",
    "Do you enjoy playing sports?",
    "Yes, I love playing basketball.",
    "What is your favorite holiday tradition?",
    "I enjoy exchanging gifts with family.",
    "How do you like to spend your weekends?",
    "I like to go on outdoor adventures.",
    "Do you prefer watching movies or reading books?",
    "I prefer reading books.",
    "What is your favorite type of food?",
    "I enjoy Mediterranean cuisine.",
    "How do you celebrate New Year's?",
    "I celebrate with a big party and fireworks.",
    "Do you like trying new recipes?",
    "Yes, I enjoy experimenting with new dishes.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you stay organized?",
    "I use a planner and keep a tidy workspace.",
    "Do you enjoy attending live music events?",
    "Yes, I love going to concerts.",
    "What is your favorite way to spend a day off?",
    "I enjoy relaxing and catching up on hobbies.",
    "How do you celebrate achievements?",
    "I treat myself to something special.",
    "Do you prefer relaxing at home or going out?",
    "I prefer relaxing at home.",
    "What is your favorite holiday destination?",
    "My favorite destination is the Caribbean.",
    "How do you stay motivated to achieve your goals?",
    "I set small milestones and reward myself.",
    "Do you like visiting new restaurants?",
    "Yes, I enjoy trying different cuisines.",
    "What is your favorite type of exercise?",
    "I enjoy cycling and swimming.",
    "How do you spend your free time?",
    "I like to read, hike, or cook.",
    "Do you prefer coffee or tea?",
    "I prefer tea.",
    "What is your favorite way to unwind after a long day?",
    "I like to watch TV or read a book.",
    "How do you stay productive throughout the day?",
    "I make a list of tasks and prioritize them.",
    "Do you enjoy spending time outdoors?",
    "Yes, I love outdoor activities.",
    "What is your favorite way to celebrate a special occasion?",
    "I celebrate with a special dinner or outing.",
    "How do you stay healthy?",
    "I exercise regularly and eat nutritious food.",
    "Do you like visiting museums?",
    "Yes, I enjoy learning about history and art.",
    "What is your favorite weekend activity?",
    "I enjoy hiking or visiting a new place.",
    "How do you like to spend your holidays?",
    "I like to travel or spend time with family.",
    "Do you enjoy trying new foods?",
    "Yes, I love discovering new dishes.",
    "What is your favorite type of music?",
    "I enjoy classical music.",
    "How do you stay active during the winter?",
    "I do indoor exercises and go for walks.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite way to relax?",
    "I like to listen to music and read.",
    "How do you celebrate the end of the year?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite hobby?",
    "I enjoy gardening and painting.",
    "How do you like to spend your weekends?",
    "I like to relax and spend time with family.",
    "Do you prefer coffee or tea in the morning?",
    "I prefer coffee.",
    "What is your favorite activity during the winter?",
    "I enjoy skiing and drinking hot cocoa.",
    "How do you stay productive?",
    "I set clear goals and manage my time well.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to spend a day off?",
    "I enjoy relaxing at home or going for a walk.",
    "How do you stay organized at work?",
    "I use a planner and keep a tidy workspace.",
    "Do you like trying new restaurants?",
    "Yes, I enjoy discovering new dining spots.",
    "What is your favorite type of food?",
    "I love Italian cuisine.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or activity.",
    "Do you enjoy playing sports?",
    "Yes, I like playing soccer and tennis.",
    "What is your favorite way to unwind?",
    "I like to listen to music or read a book.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you prefer relaxing at home or going out?",
    "I prefer relaxing at home.",
    "What is your favorite TV show?",
    "I enjoy 'Friends'.",
    "How do you celebrate your achievements?",
    "I celebrate with a treat or special activity.",
    "Do you like visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite way to spend a free day?",
    "I enjoy exploring new places or relaxing at home.",
    "How do you stay productive at work?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer staying in or going out for dinner?",
    "I like staying in and cooking at home.",
    "What is your favorite type of cuisine?",
    "I enjoy Thai food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you like trying new activities?",
    "Yes, I enjoy exploring new hobbies.",
    "What is your favorite outdoor activity?",
    "I love hiking and cycling.",
    "How do you stay focused throughout the day?",
    "I break tasks into smaller steps and take breaks.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of movie?",
    "I enjoy thriller movies.",
    "How do you like to spend your weekends?",
    "I like to relax and spend time outdoors.",
    "Do you prefer coffee or tea?",
    "I prefer tea.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate special occasions?",
    "I celebrate with special outings or treats.",
    "Do you enjoy visiting parks?",
    "Yes, I love spending time in nature.",
    "What is your favorite type of music?",
    "I enjoy jazz music.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your holidays?",
    "I like to travel and spend time with loved ones.",
    "Do you enjoy trying new foods?",
    "Yes, I love exploring different cuisines.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and visiting new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting museums?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you celebrate the end of the year?",
    "I celebrate with fireworks and a party.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite type of cuisine?",
    "I love Indian food.",
    "How do you stay productive?",
    "I set goals and use a planner to manage tasks.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite way to spend a day off?",
    "I enjoy relaxing and catching up on hobbies.",
    "How do you celebrate achievements?",
    "I treat myself to something special.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite TV show?",
    "My favorite show is 'Parks and Recreation'.",
    "How do you like to relax after a long day?",
    "I like to read a book or watch TV.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them fascinating and educational.",
    "What is your favorite outdoor activity?",
    "I love going for long walks and hiking.",
    "How do you stay organized at work?",
    "I use a planner and prioritize tasks.",
    "Do you enjoy trying new recipes?",
    "Yes, I love experimenting in the kitchen.",
    "What is your favorite type of food?",
    "I enjoy Japanese cuisine.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or event.",
    "Do you like visiting new cities?",
    "Yes, I enjoy exploring new urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy yoga and running.",
    "How do you spend your weekends?",
    "I like to relax, go hiking, or try new activities.",
    "Do you enjoy attending sports events?",
    "Yes, I love going to live sports games.",
    "What is your favorite type of weather?",
    "I like mild, sunny days.",
    "How do you stay motivated?",
    "I set personal goals and track my progress.",
    "Do you prefer reading or watching movies?",
    "I prefer reading books.",
    "What is your favorite hobby?",
    "I enjoy painting and drawing.",
    "How do you celebrate birthdays?",
    "I celebrate with a party and a special meal.",
    "Do you enjoy exploring new places?",
    "Yes, I love discovering new locations.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you stay productive during the day?",
    "I create a list of tasks and manage my time effectively.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite way to relax on a weekend?",
    "I enjoy lounging at home and reading.",
    "How do you celebrate the New Year?",
    "I celebrate with fireworks and a party.",
    "Do you like trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday?",
    "My favorite holiday is Halloween.",
    "How do you stay healthy?",
    "I eat a balanced diet and exercise regularly.",
    "Do you enjoy going to the gym?",
    "Yes, I go regularly.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you spend your holidays?",
    "I like to travel or spend time with family.",
    "Do you like visiting museums?",
    "Yes, I find them very interesting.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring nature.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer coffee or tea?",
    "I prefer tea.",
    "What is your favorite TV show?",
    "I enjoy 'The Crown'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or outings.",
    "Do you enjoy attending festivals?",
    "Yes, I love the vibrant atmosphere.",
    "What is your favorite way to unwind?",
    "I like to listen to calming music.",
    "How do you stay motivated?",
    "I set goals and reward myself for achievements.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite holiday tradition?",
    "I love baking holiday cookies with family.",
    "How do you like to spend your free time?",
    "I enjoy reading and going for walks.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of cuisine?",
    "I enjoy Mediterranean food.",
    "How do you celebrate birthdays?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of weather?",
    "I like sunny and mild weather.",
    "How do you stay active during the winter?",
    "I do indoor exercises and go for walks.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite hobby?",
    "I enjoy gardening and painting.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or event.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them inspiring.",
    "What is your favorite weekend activity?",
    "I enjoy exploring new places or relaxing at home.",
    "How do you stay focused and productive?",
    "I set goals and use a planner to stay on track.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite TV show?",
    "I enjoy 'The West Wing'.",
    "How do you like to spend your weekends?",
    "I like to relax and try new activities.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring new urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you like to spend time with family?",
    "Yes, I enjoy family gatherings.",
    "What is your favorite type of music?",
    "I enjoy indie music.",
    "How do you stay healthy?",
    "I eat well and exercise regularly.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to relax?",
    "I like to listen to music and read.",
    "How do you stay motivated?",
    "I set personal goals and track my progress.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",


    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite type of food?",
    "I enjoy sushi.",
    "How do you stay productive at work?",
    "I use a planner and set daily goals.",
    "Do you like visiting parks?",
    "Yes, I enjoy spending time outdoors.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring nature.",
    "How do you stay organized?",
    "I use a planner and make to-do lists.",
    "Do you prefer coffee or tea in the morning?",
    "I prefer coffee.",
    "What is your favorite hobby?",
    "I enjoy playing chess.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you like trying new recipes?",
    "Yes, I love experimenting in the kitchen.",
    "What is your favorite type of music?",
    "I enjoy rock music.",
    "How do you unwind after a long day?",
    "I like to read or watch TV.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to stay on track.",
    "Do you enjoy attending sports events?",
    "Yes, I love going to live games.",
    "What is your favorite way to relax?",
    "I enjoy reading and listening to music.",
    "How do you stay organized at home?",
    "I use a calendar and keep things tidy.",
    "Do you like visiting new cities?",
    "Yes, I enjoy exploring urban areas.",
    "What is your favorite type of food?",
    "I enjoy Mexican cuisine.",
    "How do you celebrate special moments?",
    "I celebrate with special activities or treats.",
    "Do you enjoy playing board games?",
    "Yes, I love playing with family and friends.",
    "What is your favorite way to unwind?",
    "I like to read a good book or listen to music.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you prefer relaxing at home or going out?",
    "I prefer relaxing at home.",
    "What is your favorite type of exercise?",
    "I enjoy pilates and jogging.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to spend your weekends?",
    "I like to relax and do activities I enjoy.",
    "How do you stay motivated?",
    "I set personal goals and track my progress.",
    "Do you prefer coffee or tea?",
    "I prefer tea.",
    "What is your favorite hobby?",
    "I enjoy crafting and DIY projects.",
    "How do you celebrate achievements?",
    "I treat myself to something special.",
    "Do you like visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite way to relax after a busy day?",
    "I enjoy listening to music and reading.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer coffee or tea in the morning?",
    "I prefer coffee.",
    "What is your favorite holiday?",
    "My favorite holiday is Easter.",
    "How do you like to spend your free time?",
    "I like to read and explore new places.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and trying new activities.",
    "How do you stay focused during work?",
    "I set clear goals and manage my time effectively.",
    "Do you like trying new recipes?",
    "Yes, I enjoy experimenting in the kitchen.",
    "What is your favorite type of food?",
    "I enjoy French cuisine.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or activity.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite type of exercise?",
    "I enjoy dancing and swimming.",
    "How do you stay motivated throughout the day?",
    "I set goals and take breaks to stay on track.",
    "Do you prefer staying in or going out for dinner?",
    "I like going out for dinner.",
    "What is your favorite TV show?",
    "My favorite show is 'Stranger Things'.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of music?",
    "I enjoy classical and jazz music.",
    "How do you unwind after a long day?",
    "I like to relax with a good book or TV show.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite hobby?",
    "I enjoy baking and cooking.",
    "How do you celebrate New Year's?",
    "I celebrate with a party and fireworks.",
    "Do you like trying new foods?",
    "Yes, I love discovering new dishes.",
    "What is your favorite type of cuisine?",
    "I enjoy Middle Eastern food.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring different urban areas.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and relaxing with family.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and trying new activities.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I like to read a good book or listen to music.",
    "How do you stay motivated throughout the day?",
    "I set clear goals and use a planner.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Italian cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you like trying new recipes?",
    "Yes, I love experimenting with new dishes.",
    "What is your favorite TV show?",
    "My favorite show is 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or outings.",
    "Do you enjoy attending live music events?",
    "Yes, I love concerts and music festivals.",
    "What is your favorite type of exercise?",
    "I enjoy yoga and running.",
    "How do you stay organized?",
    "I use a planner and make to-do lists.",
    "Do you like visiting museums?",
    "Yes, I find them very interesting.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and trying new things.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new activities.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new places.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them inspiring.",
    "What is your favorite type of music?",
    "I enjoy alternative music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to stay organized.",
    "Do you like to try new foods?",
    "Yes, I love discovering new dishes.",
    "What is your favorite type of food?",
    "I enjoy Mediterranean cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them very interesting.",


 "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and special treats.",
    "Do you enjoy attending concerts?",
    "Yes, I love live music events.",
    "What is your favorite type of exercise?",
    "I enjoy hiking and swimming.",
    "How do you stay productive?",
    "I use a planner and set daily goals.",
    "Do you prefer spending time indoors or outdoors?",
    "I enjoy both, but mostly outdoor activities.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and exploring new places.",
    "How do you stay healthy and active?",
    "I exercise regularly and eat a balanced diet.",
    "Do you enjoy visiting cultural sites?",
    "Yes, I find them very interesting.",
    "What is your favorite way to unwind?",
    "I enjoy listening to music and reading.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner to manage tasks.",
    "Do you prefer coffee or tea?",
    "I prefer coffee.",
    "What is your favorite TV show?",
    "My favorite show is 'The Office'.",
    "How do you celebrate special occasions?",
    "I celebrate with special activities or treats.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite holiday?",
    "My favorite holiday is Christmas.",
    "How do you like to spend your free time?",
    "I enjoy reading and exploring new places.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite type of food?",
    "I enjoy Thai cuisine.",
    "How do you celebrate your achievements?",
    "I celebrate with a special treat or outing.",
    "Do you enjoy visiting new cities?",
    "Yes, I love exploring urban areas.",
    "What is your favorite type of exercise?",
    "I enjoy running and yoga.",
    "How do you stay motivated?",
    "I set clear goals and track my progress.",
    "Do you like attending sports events?",
    "Yes, I enjoy live sports games.",
    "What is your favorite type of music?",
    "I enjoy indie and alternative music.",
    "How do you unwind after a long day?",
    "I like to relax with a book or TV show.",
    "Do you enjoy trying new activities?",
    "Yes, I love exploring new hobbies.",
    "What is your favorite holiday tradition?",
    "I enjoy baking holiday treats with family.",
    "How do you stay productive during the day?",
    "I make a list of tasks and set deadlines.",
    "Do you prefer coffee or tea?",
    "I prefer tea in the afternoon.",
    "What is your favorite TV show?",
    "I enjoy 'Brooklyn Nine-Nine'.",
    "How do you celebrate special occasions?",
    "I celebrate with a special meal or outing.",
    "Do you enjoy attending live performances?",
    "Yes, I love concerts and theater.",
    "What is your favorite way to relax?",
    "I like to listen to calming music and read.",
    "How do you stay organized?",
    "I use a planner and to-do lists.",
    "Do you prefer staying in or going out?",
    "I prefer staying in and relaxing.",
    "What is your favorite type of cuisine?",
    "I enjoy Japanese food.",
    "How do you celebrate your birthday?",
    "I celebrate with a party and cake.",
    "Do you enjoy exploring new hobbies?",
    "Yes, I love trying new things.",
    "What is your favorite weekend activity?",
    "I enjoy hiking and spending time outdoors.",
    "How do you stay healthy?",
    "I exercise regularly and eat a balanced diet.",
    "Do you like visiting cultural sites?",
    "Yes, I find them fascinating.",
    "What is your favorite holiday?",
    "My favorite holiday is Thanksgiving.",
    "How do you like to spend your weekends?",
    "I like to relax and explore new activities.",
    "Do you prefer coffee or tea?",
    "I prefer coffee in the morning.",
    "What is your favorite hobby?",
    "I enjoy playing the guitar.",
    "How do you celebrate New Year's?",
    "I celebrate with fireworks and a party.",
    "Do you enjoy visiting art galleries?",
    "Yes, I find them very inspiring.",
    "What is your favorite type of music?",
    "I enjoy folk music.",
    "How do you unwind after a busy day?",
    "I like to read or watch TV.",
    "Do you prefer staying in or going out for dinner?",
    "I prefer staying in and cooking at home.",
    "What is your favorite holiday tradition?",
    "I love decorating for the holidays.",
    "How do you stay motivated throughout the day?",
    "I set goals and use a planner.",
    "Do you like to try new foods?",
    "Yes, I love discovering new cuisines.",
    "What is your favorite type of food?",
    "I enjoy Middle Eastern cuisine.",
]

Save Data in Vector Database

In [11]:
embedding_manager.store_embeddings_to_file(data_emb, vector_database)

Run App with correct function

In [13]:
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])

def index():

    if request.method == 'POST':
        var.input_text = request.form.get('text', '')
        var.lang = request.form.get('language')
        action = request.form.get('action')
        if len(var.input_text) > 2000:
            var.input_text = var.input_text[:1999]
        
        if not application.validate_input(var.input_text):
            var.error = messages.message["English"]
        else:
            if action == 'translate':
                    var.text = var.input_text
                    var.error = None
                    var.details = None
                    data = application.translate_with_context(var.input_text, var.lang)
                    var.translated_text = data[0]
                    var.current_details = data[1]
                    redirect(url_for('index'))


            elif action == 'show_details':
                if var.current_details == None:
                    var.error = messages.message["show_details"]
                else:
                    if var.text == var.input_text:
                        var.details = var.current_details
                    else:
                        var.details = None
                        var.error = messages.message["show_details"]
                        var.translated_text = None

            elif action == 'reset':
                var.data = None
                var.details = None
                var.current_details = None
                var.error = None
                var.input_text = ""
                var.translated_text = None
                return redirect(url_for('index'))
                    

    return render_template('index.html', input_text=var.input_text, translated_text=var.translated_text, details=var.details,selectedLang=var.lang, error=var.error )


if __name__ == '__main__':
    try:
        app.run(debug=True, use_reloader=False)
    except SystemExit:
        sys.exit(0)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Sep/2024 18:40:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2024 18:40:37] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Sep/2024 18:40:37] "GET /static/Geo%20Speak.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Sep/2024 18:40:38] "GET /static/Geo%20Speak.jpg HTTP/1.1" 304 -
